# [WB2] PD2 Kacper Kurowski

Wczytajmy przydatne paczki

In [80]:
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import GradientBoostingClassifier #wybrany model

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

import scipy, skopt

from sklearn.metrics import f1_score

## Preprocessing

Dokonujemy preprocessing danych. Musiał być częściowo zmieniony względem poprzedniej pracy domowej.

In [2]:
data = pd.read_csv("dataset_diabetes/diabetic_data.csv")

In [3]:
data2 = data.replace('?', pd.NA)

In [4]:
data2['race'] = data2['race'].fillna('Unknown')
data2['payer_code'] = data2['payer_code'].fillna('Unknown')
data2['medical_specialty'] = data2['medical_specialty'].fillna('Unknown')

data2['diag_1'] = data2['diag_1'].fillna('Unknown')
data2['diag_2'] = data2['diag_2'].fillna('Unknown')
data2['diag_3'] = data2['diag_3'].fillna('Unknown')

In [5]:
data2['Count_nbr'] = data2.groupby('patient_nbr')['patient_nbr'].transform('count')

In [6]:
data2.drop('encounter_id', axis=1, inplace=True)
data2.drop('patient_nbr', axis=1, inplace=True)
data2.drop('weight', axis=1, inplace=True)

In [7]:
dictionary = {
    '[0-10)' : 5,
    '[10-20)' : 15,
    '[20-30)' : 25,
    '[30-40)' : 35,
    '[40-50)' : 45,
    '[50-60)' : 55,
    '[60-70)' : 65,
    '[70-80)' : 75,
    '[80-90)' : 85,
    '[90-100)' : 95
}

In [8]:
data2['age'] = data2['age'].map(dictionary)

In [10]:
y_transf = {
    'NO' : 0,
    '>30': 1,
    '<30': 1
}

In [11]:
data2["readmitted"] = data2["readmitted"].map(y_transf)

In [16]:

ohe_columns = data2.select_dtypes(exclude='number').columns.tolist()
ohe_columns.extend(['admission_type_id', 'discharge_disposition_id', 'admission_source_id'])


In [20]:
y = data2["readmitted"]
data2.drop("readmitted", inplace=True, axis=1)

In [21]:
encoder = ColumnTransformer([
        ('one_hot_encoder', OneHotEncoder(drop='first'), ohe_columns),
    ],
    remainder='passthrough'
)
X = encoder.fit_transform( data2)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


## Wytenowane modele

### Domyślny GradBoostClass

In [26]:
gbc_def = GradientBoostingClassifier()
gbc_def.fit( X_train, y_train)

GradientBoostingClassifier()

In [ ]:
gbc_def_y_pred_proba = gbc_def.predict_proba(X_test)[::,1]
gbc_def_y_pred_class = gbc_def.predict(X_test)

### GradBoostClass z własnymi hiperparam

In [27]:

gbc_min = GradientBoostingClassifier( criterion='mse', learning_rate=0.05)
gbc_min.fit( X_train, y_train)

GradientBoostingClassifier(criterion='mse', learning_rate=0.05)

In [ ]:
gbc_min_y_pred_proba = gbc_min.predict_proba(X_test)[::,1]
gbc_min_y_pred_class = gbc_min.predict(X_test)

### GradBoostClass z grid search

In [33]:
gbc_cv = GradientBoostingClassifier()

In [34]:
parameters = {
    'criterion':('friedman_mse', 'mse'),
    'learning_rate': (0.02, 0.05, 0.1,0.2,0.5 ),
    'max_depth': (2,3,4,5)
}

In [35]:
clf = GridSearchCV( gbc_cv, parameters)

In [36]:
clf.fit( X_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'criterion': ('friedman_mse', 'mse'),
                         'learning_rate': (0.02, 0.05, 0.1, 0.2, 0.5),
                         'max_depth': (2, 3, 4, 5)})

In [64]:
gbc_gri_y_pred_proba = clf.predict_proba(X_test)[::,1]
gbc_gri_y_pred_class = clf.predict(X_test)

### GradBoostClass z random search

In [47]:
parameters2 = {
    'criterion':('friedman_mse', 'mse'),
    'learning_rate': uniform(loc=0.02, scale = 0.48),
    'max_depth': (2,3,4,5)
}

In [44]:
rand_gcb = GradientBoostingClassifier()

In [48]:
rand_clf = RandomizedSearchCV(rand_gcb, parameters2, random_state=0)
search = rand_clf.fit(X_train, y_train)

In [68]:
gbc_ran_y_pred_proba = rand_clf.predict_proba(X_test)[::,1]
gbc_ran_y_pred_class = rand_clf.predict(X_test)

### GradBoostClass z Bayesian

In [53]:
bay_gcb = GradientBoostingClassifier()
opt_1 = skopt.BayesSearchCV(
        estimator=bay_gcb,
        search_spaces=parameters,
        scoring='roc_auc',
        n_jobs=4,
        n_iter=10,
        verbose=0,
        random_state=1
    )
opt_1.fit(X_train, y_train)

BayesSearchCV(estimator=GradientBoostingClassifier(), n_iter=10, n_jobs=4,
              random_state=1, scoring='roc_auc',
              search_spaces={'criterion': ('friedman_mse', 'mse'),
                             'learning_rate': (0.02, 0.05, 0.1, 0.2, 0.5),
                             'max_depth': (2, 3, 4, 5)})

In [70]:
gbc_bay_y_pred_proba = opt_1.predict_proba(X_test)[::,1]
gbc_bay_y_pred_class = opt_1.predict(X_test)

## Porównanie wyników

In [71]:
names = [
    "Default",
    "Mine",
    "Grid Search",
    "Random search",
    "Bayesian"
]
probs = [
    gbc_def_y_pred_proba,
    gbc_min_y_pred_proba,
    gbc_gri_y_pred_proba,
    gbc_ran_y_pred_proba,
    gbc_bay_y_pred_proba
]

classes = [
    gbc_def_y_pred_class,
    gbc_min_y_pred_class,
    gbc_gri_y_pred_class,
    gbc_ran_y_pred_class,
    gbc_bay_y_pred_class
]


In [76]:
print( "Wyniki: \n")
for i in range(5):
    print( names[i])
    print("Wartość ROC: " + str(metrics.roc_auc_score(y_test, probs[i])))
    print("Wartość F1: " + str(f1_score(y_test, classes[i]))+"\n")
    

Wyniki: 

Default
Wartość ROC: 0.8691661762253381
Wartość F1: 0.7742492829745325

Mine
Wartość ROC: 0.8640663938699079
Wartość F1: 0.7681063585424425

Grid Search
Wartość ROC: 0.874859654032675
Wartość F1: 0.7768154101995566

Random search
Wartość ROC: 0.8727821819527614
Wartość F1: 0.7786533941656297

Bayesian
Wartość ROC: 0.8743305068267772
Wartość F1: 0.776982417124192

